# Grouping
## Regions ↔ Boundaries

## Grouping by clustering
* Idea: embed pixels into high-dimensional space (e.g. 3-dimensions)
* Each pixel is a point
* Group together points

## k-means
* Assumption: each group is a Gaussian with different means and same standard deviation
    * $P(x_i|u_j) \propto e^{-\frac{1}{2\sigma^2} \| x_i - \mu_j \|^2}$
* Suppose we know all \mu_j. Which group should a point $x-i$ belong to?
    * The $j$ with the highest $P(x_i|u_j)$
    * = The $j$ with the smallest $\|x_i-\mu_j\|^2$

<br>

* **If means are known**
    * then trivial assignment to groups. 
    * How?
        * Assign data point to *nearest* mean


* **If means are not known**
    * What if we know a set of points from each cluster?
        * $x_{k_1},x_{k_2},...,x_{k_n}$ belong to cluster $k$
        * $\mu_k = \frac{(x_{k_1},x_{k_2},...,x_{k_n})}{n}$
    * Assignment of points to clusters in known, then finding means is easy
    * How?
        * Compute the mean of every cluster

<br>

* Given means, can assign points to clusters
* Given assignments, can computer means
* Idea: iterate

**Steps**
1. Randomly pick $k$ centers
2. Assign each point to nearest center
3. Re-estimate centers
4. Repeat

Input: set of datapoints, $k$
1. Randomly pick $k$ points as means
    * $\mu_i, i = 1,...,k$
2. For $i$ in [0,maxiters]:
    1. Assign each point to its nearest center
        * $y_i = \text{argmin}_j \| x_i - \mu_j \|^2$
    2. Re-estimate each center as mean os points assigned to it
        * $\mu_j = \frac{\sum_{i:y_i = j} x_i}{\sum_{i:y_i = j}1}$


* An objective function that must be minimized:
    * $\min_{\mu, y} \sum_i \| x_i - \mu_{y_i} \|^2$
* Every iteration of k-means takes a downward step:
    * Fixes $\mu$ and sets $y$ to minimize objective
    * Fixes $y$ and sets $\mu$ to minimize objective

## k-means on image pixels
* What is wrong?
    * Pixel position
        * Nearby pixels are likely to belong to the same object
        * Far-away pixels are likely to belong to different objects
    * How do we incorporate pixel position?
        * Instead of representing each pixel as (R,G,B), represent each pixel as (R,G,B,x,y)

### k-means on image pixels + position


### The issues with k-means
* Captures pixel similarity but
    * doesn't caputre continuity of contours
    * captures near / far relationships weakly
    * if not using position, can merge far away objects together
    * if using position, can split large regions
* Requires knowledge of k
* Can it deal with texture?

### Oversegmentation and superpixels
* What is a safe choice **if we don't know k**?
* Idea: Use large k
    * Can potentially break big objects, but will hopefully not merge unrelated objects
    * Later processing can decide which groups to merge
    * Called *superpixels*

# Segmentation
## From boundaries to segments
* k-means output typically doesn't align to boundaries
* Can we segment in a way that better aligns to boundaries?

## "Flood fill"
* Write image as a graph
    * Pixels are nodes
    * Add edge between nodes if it *doesn't cross detected boundary*
* Find *conntected components*
    * Start with a node
    * Perform DFS / BFS
    * Label all visited nodes as part of this component
    * Repeat with an unlabeled node

## Graph-based segmentation
* Connected components for going from boundaries to regions
* But problem if boundaries not closed
* Are there other graph-based algorithms that perform better?

* Construct a graph between pixels
* Edges connecting nodes have *weights*: how likely pixels are to be part of same object
    * e.g., weight = C - derivative magnitude
    * e.g., exp(- difference in color values)

<br>

* How to use this to group pixels?

## Graph *cuts*
* Each clustering *cuts* some edges and preserves others
* Example objective: choose clustering that cuts only weak edges
* *Min-cut*: choose cut that *minimizes total weight of edges cut*
* Can be done in polynomial time for *binary segmentation*: [Ford-Fulkerson algorithm](https://en.wikipedia.org/wiki/Ford–Fulkerson_algorithm)

## Graph segmentation algorithms more generally
* Pixels $x_i, \text{where}~ i = 1,...,N$
* Given number of segments $k$
* Want to figure out which segment to assign each pixel to: $y_i$
* Weight of edge between pixels $i$ and $j$: $w_{ij}$
    * (can be 0 for pixels not connected)

$\min_y \sum_{ij} w_{ij} \mathrm{I\!I} [y_i \neq y_j]$

$\min_y \sum_{ij} w_{ij} \mathrm{I\!I} [y_i \neq y_j] + \sum_i f_i(y_i)$

## Optimizing this objective
$\min_y \sum_{ij} w_{ij} \mathrm{I\!I} [y_i \neq y_j] + \sum_i f_i(y_i)$
* Reduces to min-cut for *binary segmentation*
* For $k > 2$, in general NP hard
* Approximations exist

## Even more generally
* Graph connectivity varies
* Edge weight definition varies
* Per-node cost definition varies
* Objective varies
* *Family of segmentation algorithms*

## An example output
* "Efficient interference n fully connected CRF's with Gaussian edge potentials"